# Caderno 2 - Pesquisa as queries na base atual e extrai as métricas

## 1. Carrega as bases de dados

In [1]:
import pandas as pd

PASTA_DADOS = './dados/'
# A pasta dos JURIS aqui não é a pasta original, e sim o resultado do caderno 1
PASTA_JURIS_TCU = f'{PASTA_DADOS}outputs/1_tratamento_juris_tcu/'

PASTA_RESULTADO_CADERNO = f'{PASTA_DADOS}outputs/2_pesquisa_queries_na_base_atual/'

# Carrega os arquivos 
def carrega_juris_tcu():
    doc1 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_1.csv', sep='|')
    doc2 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_2.csv', sep='|')
    doc3 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_3.csv', sep='|')
    doc4 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_4.csv', sep='|')
    doc = pd.concat([doc1, doc2, doc3, doc4], ignore_index=True)
    query = pd.read_csv(f'{PASTA_JURIS_TCU}query_tratado.csv', sep='|')
    qrel = pd.read_csv(f'{PASTA_JURIS_TCU}qrel_tratado.csv', sep='|')

    return doc, query, qrel

In [2]:
doc, query, qrel = carrega_juris_tcu()

print('********** JURIS_TCU ********** ')
print(f'Total de documentos: {len(doc)}')
print(f'Total de query: {len(query)}')
print(f'Total de qrel: {len(qrel)}')

********** JURIS_TCU ********** 
Total de documentos: 16045
Total de query: 150
Total de qrel: 2250


## 2. Pesquisa todas as queries e salva os primeiros 50 resultados

Configura a conexão com o Sorl

In [3]:
import getpass

# Autenticação e url da base
usuario_solr = getpass.getpass('Usuário Solr')
senha_solr = getpass.getpass('Senha solr')

url_solr = 'http://srv-inx-des-2a/solr/js-subbase/'

In [4]:
import requests
from requests.auth import HTTPBasicAuth

def consulta_solr(query, handler):
    params = {
        'q': query,
        'wt': 'json',  # Formato da resposta (JSON)
        'rows': 50,
        'fl': ['KEY', 'ENUNCIADO', 'score']
    }

    # Fazendo a requisição GET
    response = requests.get(f'{url_solr}{handler}', params=params, auth=HTTPBasicAuth(usuario_solr, senha_solr))
    # Verificando se a requisição foi bem sucedida
    return response.json()['response'] if response.status_code == 200 else None

Itera em todas as queries e salva no dataframe:

In [5]:
from tqdm import tqdm

def adiciona_resultados_da_pesquisa_na_lista(key_query, texto_query, lista, docs_encontrados, handler):
    for rank, doc_encontrado in enumerate(docs_encontrados, 1):
        resultado_query_doc = {
            'QUERY_KEY': key_query,
            'QUERY_TEXT': texto_query,
            'DOC_KEY': doc_encontrado['KEY'],
            'ENGINE': f'SOLR_PESQUISA_ORIGINAL_{handler}',
            'RANK': rank,
            'SCORE': doc_encontrado['score']
        }
        lista.append(resultado_query_doc)


handlers_para_pesquisar = ['select', 'selectSwan', 'selectSwanSynonym']
lista_resultado_consultas_solr = []
for index, row in tqdm(query.iterrows(), desc=f'Pesquisando {len(query)} queries'):
    key_query = row['KEY']
    texto_query = row['TEXT']

    for handler in handlers_para_pesquisar:
        resultado_para_query = consulta_solr(texto_query, handler)
        docs_encontrados = resultado_para_query['docs']
        adiciona_resultados_da_pesquisa_na_lista(key_query, texto_query, lista_resultado_consultas_solr, docs_encontrados, handler)

Pesquisando 150 queries: 150it [02:11,  1.14it/s]


In [6]:
df_resultado_consultas_solr = pd.DataFrame(lista_resultado_consultas_solr)
df_resultado_consultas_solr.head()
df_resultado_consultas_solr.to_csv(f'{PASTA_RESULTADO_CADERNO}resultado_solr_pesquisa_original.csv', sep='|', index=False)